# Reinforcement Learning on OpenAI Space Invader 

- Using **OpenAI Atari Library** (gym) to provide the environment for Space Invader
- Using **Convolutional Neural Network** (Tensorflow and Keras) to create our training model
- Using **Reinforcement Learning** (Keras-RL2) to train our program to play Space Invader

# Setting Up Our Libraries

In [ ]:
# Install the require packages here
!pip install -r requirements.txt

In [1]:
# Check it here after installation
import gym
import random as r 

# Create Deep Learning Model
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

# Apply Reinforcement Learning
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

# Set Up and Test Environment with OpenAI Gym

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
# This is for testing our environment
# Uncomment if you want to see how the game works

# episodes = 5

# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0

#     while not done:
#         env.render()
#         action = r.choice([0,1,2,3,4,5])
#         nState, reward, done, info = env.step(action)
#         score += reward
    
#     print(f'Episode: {episode} Score: {score}')

# env.close()

Episode: 1 Score: 20.0
Episode: 2 Score: 200.0
Episode: 3 Score: 75.0
Episode: 4 Score: 225.0
Episode: 5 Score: 75.0


# Create a Deep Learning Model with Keras and Tensorflow

- Build our **Deep Q Neural Network**
    - We will be create a **Convolutional Neural Network**, which will be the neural network that we will use to train our program


In [3]:
def buildModel(height, width, channels, actions):
    """
        Build our Deep Learning Model

        Args:
            height: height of the space environment
            width: width of the space environment
            channels: channels of our space environment 
            actions: actions that the agent is going to perform

        Return:
            model: A Deep Neural Network that is ready for training
    """
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    
    return model

In [4]:
# Create our model
model = buildModel(height, width, channels, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# Build Agent with Keras-RL

- Applying Keras' reinforcement learning library (Keras-RL)

## Reinforcement Learning

<img width="500" alt="portfolio_view" src="src/reinforcement-learning.jpg">

- We start with an agent, which will be our shooter in this case
- Our environment is the game itself
- Our agent will perform an action
- After the agent perform its action, it will receive a number called reward
    - Reward is also known as a Q-value, which will will be calculated by a formula called Q-function
    - Q-function captures the expected total future reward an agent in state (s) 
    - ![Insert diagram or equation](src/qFunction.png)
- The goal of the agent is to receive the most reward in each state

<img width="500" alt="portfolio_view" src="src/Space-Invaders-RL.png">




In [5]:
def buildAgent(model, actions):
    """
        Build our agent for training

        Args:
            model: our deep neural network
            actions: the action that the agent is going to perform

        Returns:
            dqn: A Deep Q-Network agent
    """
    # Set up our policy and memory
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.0, value_min=0.1, value_test=0.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    
    # Create our Agent
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return dqn

# Create our Agent
dqn = buildAgent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [6]:
# Train our model
dqn.fit(env, nb_steps=5000, visualize=False, verbose=2)

Training for 5000 steps ...
  708/5000: episode: 1, duration: 24.454s, episode steps: 708, steps per second:  29, episode reward: 110.000, mean reward:  0.155 [ 0.000, 30.000], mean action: 2.568 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1744/5000: episode: 2, duration: 1100.555s, episode steps: 1036, steps per second:   1, episode reward: 425.000, mean reward:  0.410 [ 0.000, 200.000], mean action: 2.449 [0.000, 5.000],  loss: 15.459784, mean_q: 7.616658, mean_eps: 0.876520
 2490/5000: episode: 3, duration: 1029.241s, episode steps: 746, steps per second:   1, episode reward: 365.000, mean reward:  0.489 [ 0.000, 200.000], mean action: 2.684 [0.000, 5.000],  loss: 5.944562, mean_q: 7.764208, mean_eps: 0.809515
 3904/5000: episode: 4, duration: 1749.460s, episode steps: 1414, steps per second:   1, episode reward: 390.000, mean reward:  0.276 [ 0.000, 30.000], mean action: 2.353 [0.000, 5.000],  loss: 0.874080, mean_q: 7.421928, mean_eps: 0.712315
 4588/5000: episode: 5, dur

In [7]:
# Run and test our AI
scores = dqn.test(env, nb_episodes=30, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 30 episodes ...
Episode 1: reward: 85.000, steps: 496
Episode 2: reward: 35.000, steps: 661
Episode 3: reward: 150.000, steps: 915
Episode 4: reward: 50.000, steps: 704
Episode 5: reward: 140.000, steps: 699
Episode 6: reward: 105.000, steps: 940
Episode 7: reward: 80.000, steps: 512
Episode 8: reward: 170.000, steps: 947
Episode 9: reward: 75.000, steps: 862
Episode 10: reward: 65.000, steps: 517
Episode 11: reward: 105.000, steps: 955
Episode 12: reward: 160.000, steps: 932
Episode 13: reward: 110.000, steps: 693
Episode 14: reward: 50.000, steps: 376
Episode 15: reward: 130.000, steps: 1047
Episode 16: reward: 15.000, steps: 386
Episode 17: reward: 105.000, steps: 866
Episode 18: reward: 140.000, steps: 920
Episode 19: reward: 70.000, steps: 899
Episode 20: reward: 10.000, steps: 518
Episode 21: reward: 265.000, steps: 747
Episode 22: reward: 165.000, steps: 1206
Episode 23: reward: 135.000, steps: 1211
Episode 24: reward: 105.000, steps: 707
Episode 25: reward: 50.000, 